In [1]:
from dataclasses import fields
from pandas import DataFrame
import numpy as np
import sea2025

Worksheet 1: Marginal pricing
=============================

# Objectives

Revise the notions of economic dispatch and marginal price on the simplest possible network:
* A geometric approach is available
* A linear programming formulation, applicable to general networks

# Examine input data

The format introduced in this section will be used in subsequent activities.

In [2]:
data = sea2025.data.read("data/fc1bus")


Examine the data:

In [3]:
fields(data)

(Field(name='buses',type=<class 'pandas.core.frame.DataFrame'>,default=<dataclasses._MISSING_TYPE object at 0x77e1fcc0e210>,default_factory=<dataclasses._MISSING_TYPE object at 0x77e1fcc0e210>,init=True,repr=True,hash=None,compare=True,metadata=mappingproxy({}),kw_only=False,_field_type=_FIELD),
 Field(name='generators',type=<class 'pandas.core.frame.DataFrame'>,default=<dataclasses._MISSING_TYPE object at 0x77e1fcc0e210>,default_factory=<dataclasses._MISSING_TYPE object at 0x77e1fcc0e210>,init=True,repr=True,hash=None,compare=True,metadata=mappingproxy({}),kw_only=False,_field_type=_FIELD),
 Field(name='lines',type=<class 'pandas.core.frame.DataFrame'>,default=<dataclasses._MISSING_TYPE object at 0x77e1fcc0e210>,default_factory=<dataclasses._MISSING_TYPE object at 0x77e1fcc0e210>,init=True,repr=True,hash=None,compare=True,metadata=mappingproxy({}),kw_only=False,_field_type=_FIELD),
 Field(name='offers',type=<class 'pandas.core.frame.DataFrame'>,default=<dataclasses._MISSING_TYPE objec

In [4]:
data.buses

,id,load,x,y
0,Bus1,350.0,0.0,0.0


Let's store the load/demand for later use:

In [5]:
assert data.buses.index.size == 1
load = data.buses.at[0, "load"]  # Python indexing starts at 0
load # [MW]

np.float64(350.0)

Note the `fixed_cost` column: It is **not** used in our formulation at this time.

In [6]:
data.generators

,id,bus_id,capacity,fixed_cost
0,A,Bus1,200.0,0.0
1,B,Bus1,200.0,6000.0
2,C,Bus1,200.0,8000.0


A trivial network (one bus) has no lines:

In [7]:
assert data.lines.index.size == 0
data.lines  # just column headings - no data rows

,from_bus_id,to_bus_id,capacity,reactance


In [8]:
data.offers

,generator_id,quantity,price,tranche,id
0,A,100.0,65.0,1,A/1
1,A,100.0,110.0,2,A/2
2,B,100.0,40.0,1,B/1
3,B,100.0,90.0,2,B/2
4,C,100.0,25.0,1,C/1
5,C,100.0,35.0,2,C/2


# Geometric solution

> The **Optimal Power Flow** problem is to determine the dispatch that satisfies a specified load/demand at minimum cost.
> As a by-product, the solution procedure furnishes the **marginal price** to use in settling the transactions (between generators and loads).

Take a moment to relate the `offers` table to the geometric solution shown here.

![](images/offer-stack-v1.png)

# Linear programming solution

The geometric procedure (manual/automated) isn't directly applicable to networks (with multiple buses/lines).

The [linear programming](https://en.wikipedia.org/wiki/Linear_programming) (LP) formulation implemented in the function above is laid out below.

In [9]:
import cvxpy as cp
p = cp.Variable(data.offers.index.size, name="p")  # decision variable - how much to inject?
objective = cp.Minimize(cp.sum([data.offers.at[o, "price"] * p[o] for o in data.offers.index]))  # generation cost
balance_constraint = cp.sum([p[o] for o in data.offers.index]) == load
problem = cp.Problem(
    objective,
    [
        balance_constraint,
        p >= 0,
        p <= data.offers["quantity"],
    ],
)
problem.solve(solver=cp.HIGHS, verbose=True)
assert problem.status == cp.OPTIMAL
total_cost = problem.value  # [$/h]
dispatch = p.value  # [MW]
marginal_price = -balance_constraint.dual_value  # [$/MWh] the -sign is convention-dependent

(CVXPY) Jul 22 12:35:28 PM: Your problem has 6 variables, 13 constraints, and 0 parameters.
(CVXPY) Jul 22 12:35:28 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Jul 22 12:35:28 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Jul 22 12:35:28 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Jul 22 12:35:28 PM: Your problem is compiled with the CPP canonicalization backend.
(CVXPY) Jul 22 12:35:28 PM: Compiling problem (target solver=HIGHS).
(CVXPY) Jul 22 12:35:28 PM: Reduction chain: CvxAttr2Constr -> Qp2SymbolicQp -> QpMatrixStuffing -> HIGHS
(CVXPY) Jul 22 12:35:28 PM: Applying reduction CvxAttr2Constr
(CVXPY) Jul 22 12:35:28 PM: Applying reduction Qp2SymbolicQp
(CVXPY) Jul 22 12:35:28 PM: Applying reduction QpMatrixStuffing
(CVXPY) Jul 22 12:35:28 PM: Applying reduction HIGHS
(CVXPY) Jul 22 12:35:28 PM: Finished problem compilation 

                                     CVXPY                                     
                                     v1.7.1                                    
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
-------------------------------------------------------------------------------
                                Numerical solver                               
-------------------------------------------------------------------------------
Running HiGHS 1.11.0 (git hash: 364c83a): Copyright (c) 2025 HiGHS under MIT licence terms
LP   has 13 rows; 6 cols; 18 nonzeros
Coefficient ranges:
  Matrix [1e+00, 1e+00]
  Cost   [2e+01, 1e+02]
  Bound  [0e+00, 0e+00]
  RHS    [1e+02, 4e+02]
Presolving model
1 rows, 6 cols, 6 nonzeros  0s
Dependent equations search running on 1 equations with time limit o

Verify that the results match our graphical solution (repeated here for convenience):

![](images/offer-stack-v1.png)

In [10]:
marginal_price

np.float64(65.0)

In [11]:
DataFrame({"offer": data.offers["id"].values, "dispatch": dispatch})

,offer,dispatch
0,A/1,50.0
1,A/2,-0.0
2,B/1,100.0
3,B/2,-0.0
4,C/1,100.0
5,C/2,100.0


In [12]:
assert np.isclose(total_cost, sum(data.offers["price"].values * dispatch))  # sanity check
total_cost

np.float64(13250.0)

We'll build on this LP solution when we tackle general networks (multiple buses and lines) and the Unit Commitment problem for longer planning horizons.